In [60]:
#import modules
import numpy as np

In [61]:
#loading files
seen_classes_attr = np.load('class_attributes_seen.npy')
unseen_classes_attr = np.load('class_attributes_unseen.npy')
Xtest = np.load('Xtest.npy')
Ytest = np.load('Ytest.npy')
X_seen_class = np.load('X_seen.npy',encoding='bytes',allow_pickle=True)

In [62]:
#ML assignment 6 (a)
# Mean of X_seen
mean_seen_class =[]
mean=[]

for i in range(40):
    mean=[]
    mean=np.mean(X_seen_class[i] ,axis=0)
    mean_seen_class.append(mean)

#print(np.array(mean_seen_class))

In [63]:
#similarity
similarity = np.zeros(10)
similarity = np.dot(unseen_classes_attr , seen_classes_attr.T)
#print(similarity)
#print(np.array(similarity))

In [64]:
#normalize
for i in range(10):
    sum=0
    for j in range(40):
        sum = sum + similarity[i][j]
    for j in range(40):
        similarity[i][j] = (similarity[i][j])/sum

#print(np.array(similarity))

In [65]:
#mean
mean_unseen_class=[]
mean_unseen_class = np.dot(similarity,mean_seen_class)
#print(np.array(mean_unseen_class))

In [66]:

predicted_data = np.zeros((6180, 1))
for i in range(6180):
    d = np.zeros(10)
    for j in range(10):
         d[j] = np.linalg.norm(Xtest[i] - mean_unseen_class[j] )
    min_d = np.argmin(d)
    predicted_data[i] = min_d + 1 


#print(np.array(predicted_data).shape)
#print(np.array(Ytest))

accuracy = np.sum(Ytest==predicted_data)/6180
print("Accuracy = ",accuracy*100,"%")


Accuracy =  46.89320388349515 %


In [67]:
#ML assignment 6 (b) regression 

#mean of seen classes
M = mean_seen_class

#seen class attribute
A = seen_classes_attr

#calculating w
λ = [0.01, 0.1, 1, 10, 20, 50, 100]
for item in λ:
    A_transpose = np.transpose(A)  # Transpose of A
    As = np.dot(A_transpose, A)    # A_transpose dot A
    I = np.identity(As.shape[0])   # Identity matrix of the same size as As
    λI = item * I                  # λ multiplied by the identity matrix

    W = np.linalg.inv(As + λI).dot(A_transpose).dot(M)
    #print(np.array(W).shape)

    #calculating mean for unseen classes
    ac = unseen_classes_attr

    unseen_classes = np.dot( ac,W)
    #print(np.array(unseen_classes).shape)

    predicted_data = np.zeros((6180, 1))
    for i in range(6180):
        d = np.zeros(10)
        for j in range(10):
            d[j] = np.linalg.norm(Xtest[i] - unseen_classes[j] )
        min_d = np.argmin(d)
        predicted_data[i] = min_d + 1 

    #print(np.array(predicted_data).shape)
    #print(np.array(Ytest))

    accuracy= np.sum(Ytest==predicted_data)/6180
    print("Accuracy at lambda ",item ,"=", accuracy*100,"%")

Accuracy at lambda  0.01 = 58.090614886731395 %
Accuracy at lambda  0.1 = 59.54692556634305 %
Accuracy at lambda  1 = 67.39482200647248 %
Accuracy at lambda  10 = 73.28478964401295 %
Accuracy at lambda  20 = 71.68284789644012 %
Accuracy at lambda  50 = 65.08090614886731 %
Accuracy at lambda  100 = 56.47249190938511 %


In [2]:
import numpy as np

def load_data(file_path):
    return np.load(file_path, encoding='bytes', allow_pickle=True)

def calculate_mean_vectors(data):
    mean_vectors = []
    for class_data in data:
        class_mean_vector = np.mean(class_data, axis=0)
        mean_vectors.append(class_mean_vector)
    return mean_vectors

def calculate_similarity_scores(unseen_attributes, seen_attributes):
    return np.dot(unseen_attributes, seen_attributes.T)

def normalize_similarity_scores(scores):
    normalized_scores = np.zeros_like(scores)
    for i in range(len(scores)):
        sum_scores = np.sum(scores[i])
        normalized_scores[i] = scores[i] / sum_scores
    return normalized_scores

def predict_unseen_classes(test_data, mean_unseen_vectors):
    predicted_labels = np.zeros((len(test_data), 1))
    for i in range(len(test_data)):
        distances = np.zeros(len(mean_unseen_vectors))
        for j in range(len(mean_unseen_vectors)):
            distances[j] = np.linalg.norm(test_data[i] - mean_unseen_vectors[j])
        min_distance_class = np.argmin(distances)
        predicted_labels[i] = min_distance_class + 1
    return predicted_labels

def calculate_accuracy(true_labels, predicted_labels):
    correct_predictions = np.sum(true_labels == predicted_labels)
    total_samples = len(true_labels)
    return (correct_predictions / total_samples) * 100

if __name__== "__main__":
    seen_attrs = load_data('class_attributes_seen.npy')
    unseen_attrs = load_data('class_attributes_unseen.npy')
    test_data = load_data('Xtest.npy')
    test_labels = load_data('Ytest.npy')
    seen_data = load_data('X_seen.npy')

    mean_seen_vectors = calculate_mean_vectors(seen_data)
    similarity_scores = calculate_similarity_scores(unseen_attrs, seen_attrs)
    normalized_similarity_scores = normalize_similarity_scores(similarity_scores)
    mean_unseen_vectors = np.dot(normalized_similarity_scores, mean_seen_vectors)

    predicted_labels = predict_unseen_classes(test_data, mean_unseen_vectors)
    accuracy = calculate_accuracy(test_labels, predicted_labels)
    
    print("Accuracy =", accuracy, "%")

Accuracy = 46.89320388349515 %


In [6]:
import numpy as np

def load_data(file_path):
    return np.load(file_path, encoding='bytes', allow_pickle=True)


def calculate_mean_vectors(class_data):
    mean_vectors = []
    for i in range(len(class_data)):
        class_mean = np.mean(class_data[i], axis=0)
        mean_vectors.append(class_mean)
    return mean_vectors

def calculate_accuracy(true_labels, predicted_labels):
    return np.sum(true_labels == predicted_labels) / len(true_labels)

def perform_regularized_regression(seen_attrs, unseen_attrs, test_data, test_labels, seen_data):
    # Define regularization parameter values
    lambda_values = [0.01, 0.1, 1, 10, 20, 50, 100]
    
    # Iterate over lambda values
    for lambda_val in lambda_values:
        # Prepare data
        A = seen_attrs
        A_transpose = np.transpose(A)
        As = np.dot(A_transpose, A)
        I = np.identity(As.shape[0])
        λI = lambda_val * I

        # Calculate mean vectors for seen data
        mean_seen = calculate_mean_vectors(seen_data)

        # Compute weights using regularized regression
        weights = np.linalg.inv(As + λI).dot(A_transpose).dot(mean_seen)

        # Calculate mean vectors for unseen data
        mean_unseen = np.dot(unseen_attrs, weights)

        # Predict labels for test data
        predicted_labels = np.zeros((len(test_data), 1))
        for i in range(len(test_data)):
            distances = np.zeros(10)
            for j in range(10):
                distances[j] = np.linalg.norm(test_data[i] - mean_unseen[j])
            min_distance_class = np.argmin(distances)
            predicted_labels[i] = min_distance_class + 1

        # Calculate accuracy
        accuracy = calculate_accuracy(test_labels, predicted_labels)
        print(f"Accuracy at lambda = {lambda_val} is {accuracy * 100}%")

# Load data files
seen_class_attributes = load_data('class_attributes_seen.npy')
unseen_class_attributes = load_data('class_attributes_unseen.npy')
test_data = load_data('Xtest.npy')
test_labels = load_data('Ytest.npy')
seen_class_data = load_data('X_seen.npy')

# Perform regularized regression and evaluate performance
perform_regularized_regression(seen_class_attributes, unseen_class_attributes, test_data, test_labels, seen_class_data)

Accuracy at lambda = 0.01 is 58.090614886731395%
Accuracy at lambda = 0.1 is 59.54692556634305%
Accuracy at lambda = 1 is 67.39482200647248%
Accuracy at lambda = 10 is 73.28478964401295%
Accuracy at lambda = 20 is 71.68284789644012%
Accuracy at lambda = 50 is 65.08090614886731%
Accuracy at lambda = 100 is 56.47249190938511%
